In [1]:
class HistogramFilter1D:
    
  def __init__(self, world_map, hit_weight = 0.6, miss_weight = 0.2, 
                undershoot_prob = 0.1, overshoot_prob = 0.1, exact_prob = 0.8):
    
    self.hit_weight = hit_weight
    self.miss_weight = miss_weight
    
    self.world_map = world_map
    self.world_size = len(world_map)
    self.current = self.confuse()
    
    self.undershoot_prob = undershoot_prob
    self.overshoot_prob = overshoot_prob
    self.exact_prob = exact_prob
  
  def set_current(self, prob):
    self.current = prob

  def get_current(self):
    return self.current 

  def confuse(self):
    self.current = [1 / self.world_size] * self.world_size
    return self.current
  
  def sense(self, value):
    
    def normalize(x):
      s = sum(x)
      for i in range(len(x)):
        x[i] = x[i] / s
      return x
    
    for i in range(self.world_size):
      if value == self.world_map[i]:
        self.current[i] = self.hit_weight * self.current[i]
      else:
        self.current[i] = self.miss_weight * self.current[i]
    
    self.current = normalize(self.current)
    return self.current

  def move(self, steps): # cyclical world
    
    temp = [0] * self.world_size
    
    for position in range(self.world_size):
      
      val = self.current[position]
        
      loc_exact = (position + steps) % self.world_size
      loc_over = (position + steps + 1) % self.world_size
      loc_under =  (position + steps - 1) % self.world_size
    
      temp[loc_exact] += val * self.exact_prob
      temp[loc_over] += val * self.overshoot_prob
      temp[loc_under] += val * self.undershoot_prob
    
    self.current = temp 
    return temp
      

In [2]:
def formatted_print(s, l):
    
  s = str(s)
  print('{:20}: '.format(s), end = "")

  for i in range(len(l)):
    print("{:.3f}".format(l[i]), " ", end = "")
  
  print()

In [3]:
world = ['green', 'red', 'red', 'green', 'green']
robot = HistogramFilter1D(world_map = world, 
          hit_weight = 0.6, miss_weight = 0.2,
          exact_prob = 0.8, undershoot_prob = 0.1, overshoot_prob = 0.1)

In [4]:
formatted_print("initial", robot.current)
formatted_print("sensed red" , robot.sense('red'))
formatted_print("sensed green",robot.sense('green'))

initial             : 0.200  0.200  0.200  0.200  0.200  
sensed red          : 0.111  0.333  0.333  0.111  0.111  
sensed green        : 0.200  0.200  0.200  0.200  0.200  


In [5]:
robot.set_current([0, 1, 0, 0, 0])
formatted_print("initial", robot.get_current())
formatted_print("moved one step", robot.move(1))

initial             : 0.000  1.000  0.000  0.000  0.000  
moved one step      : 0.000  0.100  0.800  0.100  0.000  


In [6]:
robot.set_current([1, 0, 0, 0, 0])
formatted_print("initial", robot.get_current())

for i in range(1000):
  robot.move(1)

formatted_print("moved 100 steps", robot.get_current())

initial             : 1.000  0.000  0.000  0.000  0.000  
moved 100 steps     : 0.200  0.200  0.200  0.200  0.200  


In [7]:
robot.set_current([0, 1, 0, 0, 0])
formatted_print("initial", robot.get_current())
formatted_print("moved one step", robot.move(1))
formatted_print("moved one step", robot.move(1))

initial             : 0.000  1.000  0.000  0.000  0.000  
moved one step      : 0.000  0.100  0.800  0.100  0.000  
moved one step      : 0.010  0.010  0.160  0.660  0.160  


In [8]:
formatted_print("initial", robot.confuse())
formatted_print("sensed red", robot.sense('red'))
formatted_print("moved one step", robot.move(1))
formatted_print("sensed green", robot.sense('green'))
formatted_print("moved one step", robot.move(1))

initial             : 0.200  0.200  0.200  0.200  0.200  
sensed red          : 0.111  0.333  0.333  0.111  0.111  
moved one step      : 0.111  0.133  0.311  0.311  0.133  
sensed green        : 0.158  0.063  0.147  0.442  0.189  
moved one step      : 0.212  0.152  0.081  0.168  0.387  


In [9]:
formatted_print("initial", robot.confuse())
formatted_print("sensed red", robot.sense('red'))
formatted_print("moved one step", robot.move(1))
formatted_print("sensed red", robot.sense('red'))
formatted_print("moved one step", robot.move(1))

initial             : 0.200  0.200  0.200  0.200  0.200  
sensed red          : 0.111  0.333  0.333  0.111  0.111  
moved one step      : 0.111  0.133  0.311  0.311  0.133  
sensed red          : 0.059  0.212  0.494  0.165  0.071  
moved one step      : 0.079  0.075  0.225  0.433  0.188  


In [10]:
robot.set_current([0, 0.5, 0.5, 0, 0])
formatted_print("initial", robot.get_current())
formatted_print("moved one step", robot.move(2))

initial             : 0.000  0.500  0.500  0.000  0.000  
moved one step      : 0.050  0.000  0.050  0.450  0.450  


In [11]:
robot.set_current([0, 0.5, 0, 0.5, 0])
formatted_print("initial", robot.get_current())
formatted_print("moved one step", robot.move(2))

initial             : 0.000  0.500  0.000  0.500  0.000  
moved one step      : 0.400  0.050  0.050  0.400  0.100  


In [12]:
robot.set_current([0, 0.25, 0, 0.75, 0])
formatted_print("initial", robot.get_current())
formatted_print("moved one step", robot.move(2))

initial             : 0.000  0.250  0.000  0.750  0.000  
moved one step      : 0.600  0.075  0.025  0.200  0.100  
